In [1]:
import json
import numpy as np
from keras import models
from keras import layers
from keras import optimizers
from keras import utils

Using TensorFlow backend.


In [2]:
with open('/vagrant/emociones.json') as file:
    input_data = json.load(file)

Convierto el formato del json a un dataset, que será un array de objetos de tipo {input: 'el texto', output: 'la etiqueta'}

In [3]:
ds_input_data = []
li_labels = list(input_data.keys())
for label in input_data.keys():
    for text in input_data[label]:
        ds_input_data.append({'input': text, 'output': label})


In [4]:
from random import shuffle
shuffle(ds_input_data)

In [5]:
import import_ipynb
import bag_of_words as bow

importing Jupyter notebook from bag_of_words.ipynb
['Del', 'salón', 'en', 'el', 'ángulo', 'oscuro', 'de', 'su', 'dueño', 'tal', 'vez', 'olvidada']
{'words': ['erase', 'un', 'hombre', 'a', 'una', 'nariz', 'pegado', 'superlativa'], 'dictionary': {'erase': 2, 'un': 1, 'hombre': 1, 'a': 1, 'una': 2, 'nariz': 2, 'pegado': 1, 'superlativa': 1}}
[2, 1, 1, 1, 1, 1, 1, 0]


In [6]:
import numpy as np

texts = [entry['input'] for entry in ds_input_data]
labels = [utils.to_categorical(li_labels.index(entry['output']), num_classes=2) for entry in ds_input_data]

dictionary = bow.extractDictionary(texts)

La función vectorize convierte un objeto de tipo {input: 'el texto', output: 'la etiqueta'} en otro de tipo de tipo {input: np.array(...), output: np.array(...)}. Es decir en una entrada y salida válida para introducir en la red neuronal.

In [7]:
def vectorize_input(text):   
    return np.array(bow.bow(text, dictionary))

In [8]:
def vectorize_output(label, n):
    return utils.to_categorical(li_labels.index(label), num_classes=n)

In [28]:
vc_inputs = np.array([vectorize_input(entry['input']) for entry in ds_input_data])
vc_inputs = vc_inputs.astype('float32')
vc_outputs = np.array([vectorize_output(entry['output'], 2) for entry in ds_input_data])
print(vc_inputs)
print(vc_outputs)


[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [33]:
input_size = vc_inputs.shape[1]
model = models.Sequential()
model.add(layers.Dense(units=18, activation='sigmoid', input_shape=(input_size,)))
model.add(layers.Dense(units=2, activation='softmax'))

In [34]:

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [35]:
model.fit(vc_inputs, vc_outputs, epochs=5, batch_size=32)

Epoch 1/5
10/10 [==============================] - 0s 32ms/step - loss: 0.7815 - acc: 0.5000
Epoch 2/5
10/10 [==============================] - 0s 133us/step - loss: 0.7626 - acc: 0.5000
Epoch 3/5
10/10 [==============================] - 0s 154us/step - loss: 0.7499 - acc: 0.5000
Epoch 4/5
10/10 [==============================] - 0s 159us/step - loss: 0.7400 - acc: 0.5000
Epoch 5/5
10/10 [==============================] - 0s 144us/step - loss: 0.7315 - acc: 0.5000


In [36]:
print(dictionary)
pruebas = ['estoy contento', 'vaya mierda']
vc_pruebas = np.array([vectorize_input(prueba) for prueba in pruebas])
print(vc_pruebas)
model.predict(vc_pruebas)

{'words': ['que', 'guay', 'peor', 'imposible', 'estoy', 'feliz', 'me', 'encuentro', 'mal', 'fatal', 'felicidad', 'vaya', 'mierda', 'esto', 'es', 'horroroso', 'contento', 'chulo'], 'dictionary': {'que': 3, 'guay': 1, 'peor': 1, 'imposible': 1, 'estoy': 3, 'feliz': 1, 'me': 1, 'encuentro': 1, 'mal': 1, 'fatal': 1, 'felicidad': 1, 'vaya': 1, 'mierda': 1, 'esto': 1, 'es': 1, 'horroroso': 1, 'contento': 1, 'chulo': 1}}
[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0]]


array([[0.65190893, 0.348091  ],
       [0.6149154 , 0.38508457]], dtype=float32)